In [2]:
import os
import sys
sys.path.append("/home/rohit/PhD_Work/GM_my_version/Graph_matching/")
from sklearn.cluster import KMeans
import networkx as nx
import numpy as np
from graph_generation.load_graphs_and_create_metadata import dataset_metadata
from graph_matching_tools.metrics import matching
import matplotlib.pyplot as plt
import scipy.io as sio

In [78]:
path_to_graph_folder = '/home/rohit/PhD_Work/GM_my_version/Graph_matching/data/simu_graph/simu_test_single_noise/0.0/noise_100,outliers_varied/'
path_to_dummy_graphs = '/home/rohit/PhD_Work/GM_my_version/Graph_matching/data/simu_graph/simu_with_dummy/0.0/noise_100,outliers_varied/0/graphs/'
path_to_groundtruth_ref = path_to_graph_folder + '/permutation_to_ref_graph.gpickle'
path_to_groundtruth = path_to_graph_folder + '/ground_truth.gpickle'

In [21]:
X_kergm = sio.loadmat(path_to_graph_folder+'/X_pairwise_kergm.mat')['full_assignment_mat']

In [22]:
graph_meta = dataset_metadata(path_to_graph_folder+'/graphs/', path_to_groundtruth_ref)

In [29]:
def get_permutation_matrix_from_dictionary(matching, g_sizes):
    """
    Create the full permutation matrix from the matching result
    :param matching: the matching result for each graph (nodes number, assignment)
    :param g_sizes: the list of the size of the different graph
    :return: the full permutation matrix
    """
    f_size = int(np.sum(g_sizes))
    res = np.zeros((f_size, f_size))

    idx1 = 0
    for i_g1 in range(len(g_sizes)):
        idx2 = 0
        for i_g2 in range(len(g_sizes)):
            match = matching["{},{}".format(i_g1, i_g2)]
            for k in match:
                res[idx1 + int(k), idx2 + match[k]] = 1
            idx2 += g_sizes[i_g2]
        idx1 += g_sizes[i_g1]
        
    np.fill_diagonal(res,1)
    return res

In [46]:
ground_truth = nx.read_gpickle(path_to_groundtruth)
res = get_permutation_matrix_from_dictionary(ground_truth, graph_meta.sizes)

In [89]:
all_graphs = [nx.read_gpickle(path_to_dummy_graphs+'/'+g) for g in np.sort(os.listdir(path_to_dummy_graphs))]
sizes_dummy = [nx.number_of_nodes(g) for g in all_graphs]

In [35]:
# f1, prec, rec = matching.compute_f1score(X_kergm,res)

In [ ]:
for graph in all_graphs:
    